In [1]:
#Installing necessary python libraries
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
!pip install flask-ngrok
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 41.0 MB/s eta 0:00:00


In [2]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
#Importing of necessary libraries
import os
import re
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


In [4]:
#Loading of model and dataset
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "fhai50032/Symptoms_to_disease_7k"
new_model = "Llama-2-7b-chat-finetune"


lora_r = 64
lora_alpha = 16
lora_dropout = 0.1



use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False


output_dir = "./NN"

num_train_epochs = 1

fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1


warmup_ratio = 0.03
group_by_length = True


save_steps = 0
logging_steps = 25


max_seq_length = None
packing = False
device_map = {"": 0}


In [5]:
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Setting up of training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Setting up of supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="query",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)


#trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7043 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/7043 [00:00<?, ? examples/s]

Loading models

In [6]:
logging.set_verbosity(logging.CRITICAL)

import spacy
import re
nlp = spacy.load("en_core_web_lg")

model_pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=800)


In [7]:
# summarization_pipe = pipeline(task="summarization")


Text generation function
Using loaded pipeline

In [8]:
def result_text_gen(prompt):
  result = model_pipe(f"<s>[INST] {prompt} [/INST]")
  req = result[0]['generated_text'][len(prompt)+18:]
  final = ''.join([str(elem) for elem in req])
  return final

Masking Function

In [9]:

def mask_pii(final):
    # Process the text

    date_patterns = [
        r'\b\d{1,2}[\/\-]\d{1,2}[\/\-]\d{2,4}\b', # Matches dates like "24/07/24" or "24-07-2024"
        r'\b\d{1,2}(st|nd|rd|th)?\s+(January|February|March|April|May|June|July|August|September|October|November|December)\b', # Matches dates like "24th July"
    ]

    date_like_words=["past","present","future","week","day","now","today","daily","monthly","yearly","yesterday","tomorrow","presently","currently","soon","instantly"]

    doc = nlp(final)

    # Create a list to hold the masked version of the text
    masked_text = []

    # Iterate over the tokens in the doc
    for token in doc:
        # Check if the token is an entity and is one of the PII types
        if token.ent_type_ in ["PERSON", "DATE","GPE","NORP"] and token.text.lower() not in date_like_words:
            # Replace the entity with [MASK]
            masked_text.append('***')
        else:
            # Otherwise, keep the original token
            masked_text.append(token.text)

    # Join the tokens back into a single string
    masked_text_str = " ".join(masked_text)

    for pattern in date_patterns:
        masked_text_str = re.sub(pattern,'***', masked_text_str)

    return masked_text_str



input domain model classifier model training



In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np

# Sample training data
questions = [
    "What is the treatment for diabetes?",
    "How does the stock market work?",
    "What are the symptoms of COVID-19?",
    "What is the capital of France?",
    "How to bake a cake?",
    "What are the side effects of aspirin?",
    "What are the side effects of ibuprofen?",
    "How do vaccines work?",
    "What are the symptoms of the flu?",
    "How is hypertension treated?",
    "What causes migraine headaches?",
    "What are the risk factors for heart disease?",
    "How can you manage diabetes with diet?",
    "What is the recovery time for a broken bone?",
    "What are the early signs of cancer?",
    "How do you treat a bacterial infection?",
    "What are common treatments for anxiety?",
    "How does chemotherapy work?",
    "What is the purpose of a colonoscopy?",
    "What are the benefits of physical therapy?",
    "How do you perform CPR?",
    "What is the difference between a cold and the flu?",
    "How is asthma diagnosed?",
    "What are the symptoms of a stroke?",
    "How do you prevent skin cancer?",
    "What is the treatment for a sprained ankle?",
    "What is the capital of Australia?",
    "How do you bake chocolate chip cookies?",
    "What is the tallest mountain in the world?",
    "How does the internet work?",
    "What is the history of the French Revolution?",
    "How do you play the guitar?",
    "What are the best practices for investing in stocks?",
    "How does photosynthesis work in plants?",
    "What is the plot of 'To Kill a Mockingbird'?",
    "How do you solve a Rubik's cube?",
    "What are the rules of soccer?",
    "How do you install a software program on a computer?",
    "What is the significance of the Mona Lisa painting?",
    "How do you make a paper airplane?",
    "What are the basic principles of quantum mechanics?",
    "How does a car engine work?",
    "What is the process of brewing beer?",
    "How do you plan a budget for a household?",
    "What are the benefits of learning a second language?",
    "How do you write a business plan?"
]

labels = [
    'medical', 'non-medical', 'medical', 'non-medical', 'non-medical', 'medical',
    'medical', 'medical', 'medical', 'medical', 'medical', 'medical', 'medical',
    'medical', 'medical', 'medical', 'medical', 'medical', 'medical', 'medical',
    'medical', 'medical', 'medical', 'medical', 'medical', 'non-medical', 'non-medical',
    'non-medical', 'non-medical', 'non-medical', 'non-medical', 'non-medical', 'non-medical',
    'non-medical', 'non-medical', 'non-medical', 'non-medical', 'non-medical', 'non-medical',
    'non-medical', 'non-medical', 'non-medical', 'non-medical', 'non-medical', 'non-medical',
    'non-medical'
]


# Vectorize the questions
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(questions)

# Train a simple classifier
classifier = MultinomialNB()
classifier.fit(X, labels)



MultinomialNB()

In [11]:
def is_medical_question(question):
    X_test = vectorizer.transform([question])
    prediction = classifier.predict(X_test)
    return prediction[0] == 'medical'

def get_answer(question):
    return is_medical_question(question)

Output domain processing

In [12]:
medical_keywords = [
    "symptoms", "treatment", "medication", "diagnosis", "anatomy", "physiology",
    "disease", "illness", "condition", "virus", "bacteria", "infection",
    "drug", "therapy", "surgery", "prognosis", "prevention", "cure",
    "cell", "tissue", "organ", "system", "body", "health",
    "pain", "fever", "cough", "rash", "nausea", "vomiting", "diarrhea",
    "blood pressure", "heart rate", "blood sugar", "cholesterol",
    "mental health", "depression", "anxiety", "stress",
    "vaccine", "immunization", "antibody",
    "research", "clinical trial"
]

def is_medical_response(text):
  """Checks if the text contains any of the medical keywords, ignoring capitalization."""
  text_lower = text.lower()
  for keyword in medical_keywords:
    if keyword in text_lower:
      return True
  return False

def return_response(text):
  if is_medical_response(text):
    return text
  else:
    text="It appears your question falls outside the scope of my expertise, which is firmly rooted in the medical field. While I'm constantly learning and expanding my knowledge base, I wouldn't want to provide any information that could potentially be misleading or inaccurate for non-medical topics. "
    return text


Function without any summarization and domain check
**It runs for 7:30** for

"Patient: John Smith
DOB: 03/15/1978
Date of Encounter: 05/16/2024
Chief Complaint: Lower back pain

History of Present Illness:
Mr. Smith presents today complaining of severe lower back pain that began two days ago after lifting a heavy box at work. He describes the pain as sharp and stabbing, radiating down his left leg to his foot. He rates the pain as 8/10 on the pain scale and notes that it is exacerbated by movement and relieved slightly by rest. He denies any bladder or bowel dysfunction.

Past Medical History:

   Hypertension
   Hyperlipidemia
   Chronic low back pain

Medications:

  Lisinopril 10 mg daily
  Simvastatin 20 mg daily
   Naproxen 500 mg PRN for pain

Allergies:

   None

Social History:
Mr. Smith is married and works as a construction worker. He smokes one pack of cigarettes per day and drinks alcohol occasionally on weekends.

Family History:
Father had a history of myocardial infarction at age 55. Mother is alive with hypertension and osteoarthritis.

Physical Examination:

  Vital Signs: BP 140/90 mmHg, HR 80 bpm, RR 16/min, Temp 98.6°F
  General: Alert and oriented, in moderate distress due to pain
  Cardiovascular: Regular rate and rhythm, no murmurs
  Respiratory: Clear to auscultation bilaterally
  Abdomen: Soft, non-tender, non-distended
  Musculoskeletal: Decreased range of motion in lumbar spine, tenderness over lower lumbar region, positive straight leg raise on left

PDF RESPONSE without preprocessing

In [13]:
def pdf_response(prompt):
  final_prompt = "you are an expert doctor and help me with the following data " + prompt + " Give solution with the data provided ."
  unmasked_result = result_text_gen(final_prompt)
  masked_result = mask_pii(unmasked_result)
  return masked_result

Input preprocessing

In [14]:
# def pdf_response(prompt):

# #  prompt = [text.lower() for text in prompt]
# #  summarized_text = summarization_pipe(prompt)
# #   if ( get_answer(summarized_text))
#   if ( get_answer(prompt))
#     final_prompt = "you are an expert doctor and help me with the following data " + summarized_text + " Give solution with the data provided ."
#     unmasked_result = result_text_gen(final_prompt)
#     masked_result = mask_pii(unmasked_result)
#     return masked_result
#   else:
#     return "Sorry I am unable to answer this"

Output preprocessing

In [15]:
# def pdf_response(prompt):
#   # Commented out lines for potential future use
#   # prompt = [text.lower() for text in prompt]
#   # summarized_text = summarization_pipe(prompt)

#   final_prompt = "you are an expert doctor and help me with the following data " + prompt + " Give solution with the data provided ."
#   unmasked_result = result_text_gen(final_prompt)
#   text = return_response(unmasked_result)

#   if (text == unmasked_result):
#     masked_result = mask_pii(unmasked_result)
#     return masked_result
#   else:
#     return text


FORM RESPONSE FUNCTION without preprocessing


In [25]:
def form_response(prompt):

  final_prompt = "you are an expert doctor and help me with the following data " + prompt + "diagnosis with the data provided ."
  # unmasked_result = result_text_gen(prompt)
  unmasked_result = result_text_gen(final_prompt)
  masked_result = mask_pii(unmasked_result)
  return masked_result

input preprocessing


In [17]:
# def form_response(prompt):
#   if (is_medical_question(prompt)):
#     final_prompt = "you are an expert doctor and help me with the following data " + prompt + "diagnosis with the data provided ."
#     unmasked_result = result_text_gen(final_prompt)
#     masked_result = mask_pii(unmasked_result)
#     return masked_result
#   else:
#     return "Sorry I am unable to answer this"

Output preprocessing


In [18]:
# def form_response(prompt):

#   final_prompt = "you are an expert doctor and help me with the following data " + prompt + "diagnosis with the data provided ."
#   unmasked_result = result_text_gen(final_prompt)
#   text=return_response(unmasked_result)
#   if(text == unmasked_result):
#     masked_result = mask_pii(unmasked_result)
#     return masked_result
#   else:
#     return text




In [27]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

port = 5000

app = Flask(__name__)
ngrok.kill()
ngrok.set_auth_token('2gaFRqfXKFcKXy86leLgQMPoZIf_4VssYvnQY65Za8NFn4Dnx') #put your own auth token from ngrok!
public_url = ngrok.connect(port).public_url

@app.route('/pdfrequest', methods=['POST'])
def process_pdfdata():
    data = request.json
    prompt = data.get('prompt')
    print('I WAS HERE')
    print(prompt)
    completion = pdf_response(prompt)
    print('I WAS HERE NEXT')
    return jsonify({'completion': completion})

@app.route('/formrequest', methods=['POST'])
def process_formdata():
    print('CALLED')
    data = request.json
    prompt = data.get('prompt')
    print(prompt)
    completion = form_response(prompt)
    print('DONE CALL')
    print(completion[len(prompt)+7:])
    return jsonify({'completion': completion})

print(public_url)
# Run the Flask app on the specified port
app.run(port = port)

Exception in thread Thread-28 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 139, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)


https://95bb-35-247-11-142.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/May/2024 02:46:09] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/May/2024 02:47:44] "POST /formrequest HTTP/1.1" 200 -


DONE CALL
   As an expert doctor , I would like to know more information to make an accurate diagnosis . Based on the data provided , I can not make a definitive diagnosis of your sore throat . Here are some possible causes of a sore throat that I would like to explore further : 

 1 . Viral infection : A sore throat is often caused by a viral infection such as the common cold or influenza . 
 2 . Bacterial infection : A sore throat can also be caused by a bacterial infection such as strep throat , which is caused by the bacterium Streptococcus pyogenes . 
 3 . Allergies : Allergies can cause postnasal drip , which can lead to a sore throat . 
 4 . Environmental factors : Exposure to pollutants , smoke , or irritants can cause a sore throat . 
 5 . Acid reflux : Gastroesophageal reflux disease ( GERD ) can cause a sore throat , especially at night , as stomach acid flows back up into the esophagus and irritates the throat . 
 6 . Dry air : Dry air can cause a sore throat , especially i

In [20]:
x_crct="""Patient: John Smith
DOB: 03/15/1978
Date of Encounter: 05/16/2024
Chief Complaint: Lower back pain

History of Present Illness:
Mr. Smith presents today complaining of severe lower back pain that began two days ago after lifting a heavy box at work. He describes the pain as sharp and stabbing, radiating down his left leg to his foot. He rates the pain as 8/10 on the pain scale and notes that it is exacerbated by movement and relieved slightly by rest. He denies any bladder or bowel dysfunction.

Past Medical History:

    Hypertension
    Hyperlipidemia
    Chronic low back pain

Medications:

    Lisinopril 10 mg daily
    Simvastatin 20 mg daily
    Naproxen 500 mg PRN for pain

Allergies:

    None

Social History:
Mr. Smith is married and works as a construction worker. He smokes one pack of cigarettes per day and drinks alcohol occasionally on weekends.

Family History:
Father had a history of myocardial infarction at age 55. Mother is alive with hypertension and osteoarthritis.

Physical Examination:

    Vital Signs: BP 140/90 mmHg, HR 80 bpm, RR 16/min, Temp 98.6°F
    General: Alert and oriented, in moderate distress due to pain
    Cardiovascular: Regular rate and rhythm, no murmurs
    Respiratory: Clear to auscultation bilaterally
    Abdomen: Soft, non-tender, non-distended
    Musculoskeletal: Decreased range of motion in lumbar spine, tenderness over lower lumbar region, positive straight leg raise on left



    Summarize the following notes and do not reveal any sensitive data.
    give me assessment and plan for the next steps to be considered"""

In [21]:
x_wrong="""I'm interested in learning more about the concept of decentralization in Web3. , particularly in terms of data ownership, censorship resistance, and security?  Are there any real-world examples of how decentralization is being implemented in Web3 projects?"""

In [22]:
# y=pdf_response(x_crct)
# print(y)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# y=pdf_response(x_wrong)
# print(y)

In [ ]:
# y=form_response(x_crct)
# print(y)

In [ ]:
# y=form_response(x_wrong)
# print(y)